In [15]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.utils as vutils

# DCGAN Settings
lr = 0.0001
beta1 = 0.5
nz = 100  # latent vector size
ngf = 64  # generator feature map size
ndf = 64  # discriminator feature map size
num_epochs = 50
image_size = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) Defined Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

# 2) Defined Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

def train_dcgan(real_dir="RealOnly"):
    """
    This script trains DCGAN only on real images in real_dir to learn how to generate real-like images.
    The discriminator is trained to differentiate real vs. generated images.
    """
    # Created dataset
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])
    real_dataset = datasets.ImageFolder(root=real_dir, transform=transform)
    real_loader = DataLoader(real_dataset, batch_size=64, shuffle=True)

    netG = Generator().to(device)
    netD = Discriminator().to(device)

    criterion = nn.BCELoss()
    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1,0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1,0.999))

    fixed_noise = torch.randn(64, nz, 1, 1, device=device)

    for epoch in range(num_epochs):
        for i, (data, _) in enumerate(real_loader):
            netD.zero_grad()
            real = data.to(device)
            b_size = real.size(0)
            label = torch.full((b_size,), 1.0, dtype=torch.float, device=device)
            
            output = netD(real).view(-1)
            errD_real = criterion(output, label)
            errD_real.backward()

            # Training with fake
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            fake = netG(noise)
            label.fill_(0.0)
            output = netD(fake.detach()).view(-1)
            errD_fake = criterion(output, label)
            errD_fake.backward()
            optimizerD.step()

            errD = errD_real + errD_fake

            # Updating G
            netG.zero_grad()
            label.fill_(1.0) 
            output = netD(fake).view(-1)
            errG = criterion(output, label)
            errG.backward()
            optimizerG.step()

            if i % 50 == 0:
                print(f"Epoch [{epoch}/{num_epochs}] Step [{i}/{len(real_loader)}] "
                      f"Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}")

    # Save final models
    torch.save(netG.state_dict(), "dcgan_G.pt")
    torch.save(netD.state_dict(), "dcgan_D.pt")
    print("DCGAN training complete. Models saved.")

if __name__ == "__main__":
    # Provide a directory "RealOnly/" with subfolder e.g. "class_for_real" or any single label
    # so ImageFolder can load it. All images in that folder are real.
    train_dcgan(real_dir=r"C:\Users\gupta\Downloads\induction-task\Data\Train")

Epoch [0/50] Step [0/13] Loss_D: 1.4209 Loss_G: 1.7722
Epoch [1/50] Step [0/13] Loss_D: 0.3055 Loss_G: 4.7248
Epoch [2/50] Step [0/13] Loss_D: 0.1030 Loss_G: 6.1856
Epoch [3/50] Step [0/13] Loss_D: 0.0796 Loss_G: 6.9115
Epoch [4/50] Step [0/13] Loss_D: 0.0780 Loss_G: 7.9031
Epoch [5/50] Step [0/13] Loss_D: 0.0429 Loss_G: 7.4866
Epoch [6/50] Step [0/13] Loss_D: 0.0813 Loss_G: 10.6766
Epoch [7/50] Step [0/13] Loss_D: 0.0064 Loss_G: 15.2476
Epoch [8/50] Step [0/13] Loss_D: 0.0232 Loss_G: 14.1216
Epoch [9/50] Step [0/13] Loss_D: 0.0146 Loss_G: 10.1478
Epoch [10/50] Step [0/13] Loss_D: 0.0222 Loss_G: 9.2741
Epoch [11/50] Step [0/13] Loss_D: 0.0076 Loss_G: 8.8807
Epoch [12/50] Step [0/13] Loss_D: 0.0118 Loss_G: 6.6173
Epoch [13/50] Step [0/13] Loss_D: 0.0056 Loss_G: 11.8940
Epoch [14/50] Step [0/13] Loss_D: 0.9055 Loss_G: 10.3035
Epoch [15/50] Step [0/13] Loss_D: 0.2276 Loss_G: 6.4891
Epoch [16/50] Step [0/13] Loss_D: 0.2103 Loss_G: 4.4652
Epoch [17/50] Step [0/13] Loss_D: 0.1661 Loss_G: 4.4

In [16]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import pandas as pd
import glob
from PIL import Image

class DCGAN_Discriminator_Classifier(nn.Module):
    def __init__(self, pretrained_path=None):
        super(DCGAN_Discriminator_Classifier, self).__init__()
        # SAME AS DISCRIMINATOR
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )
    # Using a classifier layer to map out results to.
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(512, 2)
        )
    #Using pretrained weights from discriminator trained earlier and extracting weights and features from it.                    
  
        if pretrained_path is not None:                    
            pretrained_dict = torch.load(pretrained_path, map_location="cpu")
            own_dict = self.state_dict()
            for k, v in pretrained_dict.items():
                if k in own_dict and k.startswith('main.'):
                    # example: "main.0.weight" in the old model => "features.0.weight" if indices align
                    new_k = k.replace("main.", "features.")
                    own_dict[new_k] = v
            self.load_state_dict(own_dict, strict=False)
            print("Loaded partial pretrained discriminator weights (feature layers).")

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

def train_dcgan_classifier(
    train_dir="Train", 
    num_epochs=20,
    batch_size=16,
    lr=1e-4,
    pretrained_d="dcgan_D.pt",
    out_model="dcgan_classifier.pt"
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([
        transforms.Resize((64,64)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

    dataset = datasets.ImageFolder(train_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # 2 classes: dataset.class_to_idx => { 'AI':0, 'Real':1 } (just as an example)
    model = DCGAN_Discriminator_Classifier(pretrained_path=pretrained_d).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for i, (imgs, labels) in enumerate(loader):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    # Saving the trained script
    torch.save(model.state_dict(), out_model)
    print(f"DCGAN-based classifier saved at {out_model}")


In [17]:
def inference_dcgan_classifier(
    model_path="dcgan_classifier.pt",
    test_dir="Test",
    output_csv="submission_dcgan.csv"
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([
        transforms.Resize((64,64)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

    # Loading model
    model = DCGAN_Discriminator_Classifier(pretrained_path=None).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Inference
    import glob
    from PIL import Image
    image_paths = sorted(glob.glob(os.path.join(test_dir, "*.*")))
    results = []
    with torch.no_grad():
        for img_path in image_paths:
            img_id = os.path.splitext(os.path.basename(img_path))[0]
            img = Image.open(img_path).convert("RGB")
            x = transform(img).unsqueeze(0).to(device)
            outputs = model(x)
            _, predicted = outputs.max(1)
            label_str = "AI" if predicted.item() == 0 else "Real"
            results.append([img_id, label_str])

    df = pd.DataFrame(results, columns=["Id","Label"])
    df.to_csv(output_csv, index=False)
    print(f"Inference complete. Saved results to {output_csv}")

In [24]:


if __name__ == "__main__":
    # 1) Train DCGAN-based classifier
    train_dcgan_classifier(
        train_dir=r"C:\Users\gupta\Downloads\induction-task\Data\Train",     # your /AI, /Real subfolders
        num_epochs=20,
        batch_size=16,
        lr=1e-4,
        pretrained_d="dcgan_D.pt",  
        out_model="dcgan_classifier.pt"
    )

    # 2) Inference
    inference_dcgan_classifier(
        model_path="dcgan_classifier.pt",
        test_dir=r"C:\Users\gupta\Downloads\induction-task\Data\Test",
        output_csv="submission_dcgan1.csv"
    )

C:\Users\gupta\AppData\Local\Temp\ipykernel_38896\1398977366.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(pretrained_path, map_location=

Loaded partial pretrained discriminator weights (feature layers).
Epoch [1/20] Loss: 0.2535 Acc: 0.8964
Epoch [2/20] Loss: 0.1085 Acc: 0.9638
Epoch [3/20] Loss: 0.0844 Acc: 0.9725
Epoch [4/20] Loss: 0.0452 Acc: 0.9875
Epoch [5/20] Loss: 0.0394 Acc: 0.9888
Epoch [6/20] Loss: 0.0295 Acc: 0.9925
Epoch [7/20] Loss: 0.0279 Acc: 0.9925
Epoch [8/20] Loss: 0.0129 Acc: 0.9988
Epoch [9/20] Loss: 0.0112 Acc: 1.0000
Epoch [10/20] Loss: 0.0330 Acc: 0.9900
Epoch [11/20] Loss: 0.0196 Acc: 0.9938
Epoch [12/20] Loss: 0.0177 Acc: 0.9938
Epoch [13/20] Loss: 0.0076 Acc: 0.9988
Epoch [14/20] Loss: 0.0075 Acc: 0.9988
Epoch [15/20] Loss: 0.0112 Acc: 0.9963
Epoch [16/20] Loss: 0.0062 Acc: 1.0000
Epoch [17/20] Loss: 0.0095 Acc: 0.9975
Epoch [18/20] Loss: 0.0100 Acc: 0.9975
Epoch [19/20] Loss: 0.0091 Acc: 1.0000
Epoch [20/20] Loss: 0.0103 Acc: 1.0000
DCGAN-based classifier saved at dcgan_classifier.pt


C:\Users\gupta\AppData\Local\Temp\ipykernel_38896\4008440749.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

Inference complete. Saved results to submission_dcgan2.csv
